# 6월8일 과제:

* contacts.csv를 읽어들여 DataFrame 객체인 contacts로 만든다.
* 모든 열의 데이터에서 누락 값을 적절히 처리한다.
* 모든 열의 이름에서 공백문자를 제거한다.
* 성명’ 열의 데이터에서 공백문자를 제거한다.
* 사무실’ 열의 이름을 ‘사무실 전화번호’로 변경한다.
* 사무실’ 열의 데이터 앞에 ’(02) 870-’ 문자열을 연결한다.
* 담당업무’열을 parsing 하여 dummy 데이터로 만든다.


In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import re

In [2]:
contact = pd.read_csv('contacts.csv',encoding='euc-kr')
contact.head()

성 명   사무실          휴 대 폰                       담당업무
0  윤 택 상  5810  010-9359-6093                경영지원부 업무 총괄
1  이 동 한  5811  010-5578-4007                 인사, 교육, 총무
2  이 소 영  5813  010-9542-0313           세무회계, 자금관리, 마일리지
3  백 승 민  5812  010-9241-8185  급여, 4대보험, 퇴직금, 외부연구과제신청경비
4  송 승 미  5814  010-4255-8817                 회계결산, 정보공시

### 전처리1
* 컬럼 사무실 -> 유선번호 로 rename

In [3]:
contact.rename(columns={'사무실':'유선번호'},inplace=True)
contact.head()

성 명  유선번호          휴 대 폰                       담당업무
0  윤 택 상  5810  010-9359-6093                경영지원부 업무 총괄
1  이 동 한  5811  010-5578-4007                 인사, 교육, 총무
2  이 소 영  5813  010-9542-0313           세무회계, 자금관리, 마일리지
3  백 승 민  5812  010-9241-8185  급여, 4대보험, 퇴직금, 외부연구과제신청경비
4  송 승 미  5814  010-4255-8817                 회계결산, 정보공시

### 전처리2
* 비고 칼럼을 추가한다

In [4]:
contact['비고']=np.nan
contact.head()

성 명  유선번호          휴 대 폰                       담당업무  비고
0  윤 택 상  5810  010-9359-6093                경영지원부 업무 총괄 NaN
1  이 동 한  5811  010-5578-4007                 인사, 교육, 총무 NaN
2  이 소 영  5813  010-9542-0313           세무회계, 자금관리, 마일리지 NaN
3  백 승 민  5812  010-9241-8185  급여, 4대보험, 퇴직금, 외부연구과제신청경비 NaN
4  송 승 미  5814  010-4255-8817                 회계결산, 정보공시 NaN

### 전처리3
* 유선번호 칼럼에 숫자 아닌 문자열이 들어있으면 비고 란으로 이동하고 해당 cell은 nan처리

In [5]:
# 숫자 이외 문자열이 포함된 iloc list를 반환
def list_of_contains_hangul(col):
    hangul = re.compile('[(^\u3131-\u3163\uac00-\ud7a3()]+')
    lst_contains_hangul=[i for (i, x) in enumerate (col) if hangul.search(x) != None ]
    
    return lst_contains_hangul

In [76]:
hangul1 = re.compile('[)(\u3131-\u3163\uac00-\ud7a3]+')
text=")아래@한 ! 글&("

In [77]:
m=hangul1.search(text)

In [78]:
m.groups()

()

In [6]:
lst=list_of_contains_hangul(contact['유선번호'])
for i in lst:
    contact.loc[i,'비고']=contact.loc[i,'유선번호']
    contact.loc[i,'유선번호']=np.nan
contact.iloc[lst]

성 명 유선번호          휴 대 폰                담당업무                비고
11  최 미 애  NaN  010-9094-8698  육아휴직(~ 2018.09.09)                휴직
32  김 지 선  NaN  010-5759-7879  육아휴직(~ 2018.02.28)                휴직
42  명 현 규  NaN  010-3417-1598    창업보육센터 운영 및 행정지원  043-249-0121(오송)

### 성명’ 열의 데이터에서 공백문자를 제거한다.

In [7]:
contact.rename(columns={'성 명':'성명'}, inplace=True)
contact.columns

Index(['성명', '유선번호', '휴 대 폰', '담당업무', '비고'], dtype='object')

### 모든 열의 이름에서 공백문자를 제거한다.

In [8]:
name_split=lambda x: x.replace(' ','')
contact['성명']=contact['성명'].apply(name_split)
contact.head()

성명  유선번호          휴 대 폰                       담당업무   비고
0  윤택상  5810  010-9359-6093                경영지원부 업무 총괄  NaN
1  이동한  5811  010-5578-4007                 인사, 교육, 총무  NaN
2  이소영  5813  010-9542-0313           세무회계, 자금관리, 마일리지  NaN
3  백승민  5812  010-9241-8185  급여, 4대보험, 퇴직금, 외부연구과제신청경비  NaN
4  송승미  5814  010-4255-8817                 회계결산, 정보공시  NaN

### 유선전화 열의 데이터 앞에 ’(02) 870-’ 문자열을 연결한다.

In [9]:
contact['유선번호']='(02)870-'+contact['유선번호']
contact.head()

성명          유선번호          휴 대 폰                       담당업무   비고
0  윤택상  (02)870-5810  010-9359-6093                경영지원부 업무 총괄  NaN
1  이동한  (02)870-5811  010-5578-4007                 인사, 교육, 총무  NaN
2  이소영  (02)870-5813  010-9542-0313           세무회계, 자금관리, 마일리지  NaN
3  백승민  (02)870-5812  010-9241-8185  급여, 4대보험, 퇴직금, 외부연구과제신청경비  NaN
4  송승미  (02)870-5814  010-4255-8817                 회계결산, 정보공시  NaN

### 담당업무’열을 parsing 하여 dummy 데이터로 만든다.

In [10]:
business_iter = (set(x.split(',')) for x in contact.담당업무)
display(business_iter)
business = sorted(set.union(*business_iter))

<generator object <genexpr> at 0x0000000007FDBF68>

In [11]:
business

[' 4대보험',
 ' BI운영',
 ' BK21플러스',
 ' IP전략',
 ' 개인)',
 ' 검수',
 ' 계좌관리',
 ' 교내 학술활동지원사업(산단)',
 ' 교내연구과제',
 ' 교내연구교원지원사업',
 ' 교내학술활동지원사업(교비)',
 ' 교육',
 ' 교육부',
 ' 국제',
 ' 국토부',
 ' 규정',
 ' 그 외 정부부처',
 ' 기술사업화',
 ' 기술이전(산업자문)',
 ' 논문업적관리',
 ' 농림부',
 ' 마일리지',
 ' 미래부 산하',
 ' 법인카드',
 ' 보건복지부(한국보건산업진흥원)',
 ' 비영리기관',
 ' 서무',
 ' 서무 ',
 ' 세무(원천세)',
 ' 세종발전연구회',
 ' 엑셀러레이팅',
 ' 연구교수 관리',
 ' 연구기자재',
 ' 연구비법인카드',
 ' 예산',
 ' 외부연구과제신청경비',
 ' 외자)',
 ' 원천)',
 ' 인사 및 경영기획',
 ' 일반기업체',
 ' 입주기업지원',
 ' 자금관리',
 ' 자회사 경영지원',
 ' 저작권',
 ' 정보공시',
 ' 정부사업 관리',
 ' 중소기업청',
 ' 지식재산권',
 ' 총무',
 ' 출납',
 ' 캠퍼스CEO육성사업)',
 ' 통계',
 ' 퇴직금',
 ' 퇴직금 등)',
 ' 퇴직금)',
 ' 투자 및 컨설팅',
 ' 평생교육원',
 ' 학생인건비',
 ' 한국산업기술진흥원(지역과제)',
 ' 한국산업기술평가관리원',
 ' 한국에너지기술평가원',
 ' 한국연구재단(신진',
 ' 한국연구재단(인문사회)',
 ' 해수부',
 ' 환경부',
 'BI운영',
 'LINC+사업단 파견',
 'SK청년비상프로그램(창업진흥원)',
 '가족회사관리',
 '결산 및 세무',
 '경영지원부 업무 총괄',
 '과제관리 (맞춤형사업',
 '구매',
 '구매(연구',
 '구매(운영비)',
 '국내특허 관리',
 '급여',
 '기술발굴(BT)',
 '기술발굴(IT)',
 '기술발굴(NT)',
 '기술사업부 업무 총괄',
 '기술실용화',
 '기술이전(BT)',
 

In [12]:
dummies = DataFrame(np.zeros((len(contact), len(business))), columns=business)

In [13]:
for i, business in enumerate(contact.담당업무):
    #print(business)
    dummies.iloc[i].loc[business.split(',')] = 1
dummies.tail(10)

4대보험   BI운영   BK21플러스   IP전략   개인)   검수   계좌관리   교내 학술활동지원사업(산단)  \
37    0.0    0.0       0.0    0.0   0.0  0.0    0.0               0.0   
38    0.0    0.0       1.0    0.0   0.0  0.0    0.0               1.0   
39    0.0    0.0       0.0    0.0   1.0  0.0    0.0               0.0   
40    0.0    0.0       0.0    0.0   0.0  0.0    0.0               0.0   
41    0.0    0.0       0.0    0.0   0.0  0.0    0.0               0.0   
42    0.0    0.0       0.0    0.0   0.0  0.0    0.0               0.0   
43    0.0    0.0       0.0    0.0   0.0  0.0    0.0               0.0   
44    0.0    0.0       0.0    0.0   0.0  0.0    0.0               0.0   
45    0.0    0.0       0.0    0.0   0.0  0.0    0.0               0.0   
46    0.0    0.0       0.0    0.0   0.0  0.0    0.0               0.0   

     교내연구과제   교내연구교원지원사업  ...   창업보육센터 업무총괄  창업보육센터 운영 및 행정지원  창업중심대학 사업운영  \
37      0.0          0.0  ...           0.0               0.0          0.0   
38      0.0          0.0  ...           0.0               0.0          0.0   
39      0.0          0.0  ...           0.0               0.0          0.0   
40      1.0          0.0  ...           0.0               0.0          0.0   
41      0.0          0.0  ...           0.0               0.0          0.0   
42      0.0          0.0  ...           0.0               1.0          0.0   
43      0.0          0.0  ...           0.0               0.0          0.0   
44      0.0          0.0  ...           0.0               0.0          0.0   
45      0.0          0.0  ...           0.0               0.0          0.0   
46      0.0          0.0  ...           0.0               0.0          0.0   

    투자기업 발굴 지원  한국연구재단(이공 중견)  한국연구재단(이공일반  해외특허 관리   회계  회계·재무  회계결산  
37         0.0            1.0          0.0      0.0  0.0    0.0   0.0  
38         0.0            0.0          0.0      0.0  0.0    0.0   0.0  
39         0.0            0.0          1.0      0.0  0.0    0.0   0.0  
40         0.0            0.0          0.0      0.0  0.0    0.0   0.0  
41         0.0            0.0          0.0      0.0  0.0    0.0   0.0  
42         0.0            0.0          0.0      0.0  0.0    0.0   0.0  
43         0.0            0.0          0.0      0.0  0.0    0.0   0.0  
44         0.0            0.0          0.0      0.0  0.0    1.0   0.0  
45         0.0            0.0          0.0      0.0  1.0    0.0   0.0  
46         1.0            0.0          0.0      0.0  0.0    0.0   0.0  

[10 rows x 116 columns]

In [ ]:
끄트